In [1]:
import src.data.breathe_data as breathe_data
import src.inference.long_inf_slicing as slicing
import src.models.builders as mb
import src.models.var_builders as var_builders
import src.inference.helpers as ih
from plotly.subplots import make_subplots
import src.models.helpers as mh

import pandas as pd
import numpy as np

In [2]:
df = breathe_data.load_meas_from_excel("BR_O2_FEV1_FEF2575_with_idx")

INFO:root:* Checking for same day measurements *


In [33]:
def get_consec_days(df):
    df = df.sort_values(by='Date Recorded')
    df['Prev day'] = df['Date Recorded'].shift(1)
    # Compute n days between measurements
    df['Days elapsed'] = (df['Date Recorded'] - df['Prev day'])
    # Remove nan
    df = df.dropna(subset=['Days elapsed'])

    # Get first idx where Days elapsed is greater than 1
    idx = df[df['Days elapsed'] > pd.Timedelta(days=3)].index
    print(idx)

    return df

df_tmp = df[df.ID == '101']
get_consec_days(df_tmp).iloc[:592]
# Not finished

Index([592, 718, 890, 918, 1134, 1311, 1476], dtype='int64')


,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,...,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,idx ecFEV1 (L),idx ecFEF25-75 % ecFEV1 (%),idx O2 saturation (%),Prev day,Days elapsed
1,101,2019-01-26,1.31,98,0.57,1.31,0.67,Male,173.0,53,...,97.150104,36.287474,36.287474,100.874827,43.511450,26,21,48,2019-01-25,"1 day, 0:00:00"
2,101,2019-01-27,1.31,96,0.67,1.31,0.69,Male,173.0,53,...,97.150104,36.287474,36.287474,98.816157,51.145038,26,25,46,2019-01-26,"1 day, 0:00:00"
3,101,2019-01-28,1.30,96,0.69,1.31,0.69,Male,173.0,53,...,97.150104,36.287474,36.010470,98.816157,53.076923,26,26,46,2019-01-27,"1 day, 0:00:00"
4,101,2019-01-29,1.28,98,0.60,1.30,0.69,Male,173.0,53,...,97.150104,36.010470,35.456463,100.874827,46.875000,26,23,48,2019-01-28,"1 day, 0:00:00"
5,101,2019-01-30,1.29,97,0.59,1.34,0.75,Male,173.0,53,...,97.150104,37.118484,35.733466,99.845492,45.736434,26,22,47,2019-01-29,"1 day, 0:00:00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,101,2020-09-05,1.51,97,0.87,1.51,1.04,Male,173.0,53,...,97.150104,41.827546,41.827546,99.845492,57.615894,30,28,47,2020-09-04,"1 day, 0:00:00"
589,101,2020-09-06,1.50,98,1.04,1.51,1.04,Male,173.0,53,...,97.150104,41.827546,41.550542,100.874827,69.333333,30,34,48,2020-09-05,"1 day, 0:00:00"
590,101,2020-09-07,1.50,97,0.93,1.50,1.04,Male,173.0,53,...,97.150104,41.550542,41.550542,99.845492,62.000000,30,31,47,2020-09-06,"1 day, 0:00:00"
591,101,2020-09-08,1.49,97,0.88,1.50,0.93,Male,173.0,53,...,97.150104,41.550542,41.273539,99.845492,59.060403,30,29,47,2020-09-07,"1 day, 0:00:00"


In [4]:
# Initialize variables (not used for inference)
(HFEV1, ecFEV1, AR, HO2Sat, O2SatFFA, IA, UO2Sat, O2Sat, ecFEF2575prctecFEV1) = (
    var_builders.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(
        160, 40, "Male"
    )
)

key_hfev1 = f"['{ecFEV1.name}', '{HFEV1.name}', '{AR.name}'] -> {HFEV1.name}"
key_ho2sat = f"['{O2SatFFA.name}', '{HO2Sat.name}', '{AR.name}'] -> {HO2Sat.name}"
HFEV1.set_factor_node_key(key_hfev1)
HO2Sat.set_factor_node_key(key_ho2sat)

vars = [AR, IA]
shared_vars = [HFEV1, HO2Sat]
# obs_vars = [ecFEV1.name, O2Sat.name]
# obs_vars = [ecFEV1.name, O2Sat.name, ecFEF2575prctecFEV1]
obs_vars = [ecFEV1.name]

# interconnected_AR = "_padded_identity_15"
interconnected_AR = "_shift_span_[-20;20]_samples"
# interconnected_AR = False
# ar_prior = "uniform"
ar_prior = "uniform message to HFEV1"
save = True


def infer_and_plot_for_id(df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-8):
    df_for_ID = df_for_ID.reset_index(drop=True)
    print(f"\nID: {df_for_ID.ID.iloc[0]}")
    print(f"Number of datapoints: {len(df_for_ID)}")

    height = df_for_ID.Height.iloc[0]
    age = df_for_ID.Age.iloc[0]
    sex = df_for_ID.Sex.iloc[0]
    (
        _,
        inf_alg,
        HFEV1,
        ecFEV1,
        AR,
        HO2Sat,
        O2SatFFA,
        IA,
        UO2Sat,
        O2Sat,
        ecFEF2575prctecFEV1,
    ) = mb.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(
        height, age, sex, ia_prior="breathe", ar_prior=ar_prior
    )

    df_query_res, df_res_before_convergence, shared_vars_final = (
        slicing.query_back_and_forth_across_days(
            df_for_ID,
            inf_alg,
            shared_vars,
            vars,
            obs_vars,
            diff_threshold,
            debug=False,
            max_passes=1,
            interconnect_AR=interconnected_AR,
        )
    )

    colorscale = [
        [0, "white"],
        [0.01, "red"],
        [0.05, "yellow"],
        [0.1, "cyan"],
        [0.6, "blue"],
        [1, "black"],
    ]

    obs_vars_str = ", ".join([mh.name_to_abbr(var) for var in obs_vars])
    inteconnected_AR_str = "interdays AR, " if interconnected_AR else ""
    ar_prior_str = f"AR {ar_prior}"
    passes = "back and forth"

    title = f"ID {df_for_ID.ID[0]} - obs. {obs_vars_str}, {passes}, {inteconnected_AR_str}{ar_prior_str} ({df_for_ID.Sex[0]}, {df_for_ID.Age[0]}yr, {df_for_ID.Height[0]}cm)"
    slicing.plot_posterior_validation(
        df_res_before_convergence,
        HFEV1,
        AR,
        df_for_ID,
        title,
        colorscale,
        save,
    )

    title = f"ID {df_for_ID.ID[0]} - obs. {obs_vars_str}, {inteconnected_AR_str}{ar_prior_str} ({df_for_ID.Sex[0]}, {df_for_ID.Age[0]}yr, {df_for_ID.Height[0]}cm)"
    slicing.plot_query_res(
        df_for_ID,
        ecFEV1,
        O2Sat,
        df_query_res,
        AR,
        IA,
        HFEV1,
        HO2Sat,
        title,
        colorscale,
        save,
    )
    return df_query_res, df_res_before_convergence, shared_vars_final


# interesting_ids = [
#     "132",
#     "146",
#     "177",
#     "180",
#     "202",
#     "527",
#     "117",
#     "131",
#     "134",
#     "191",
#     "139",
#     "253",
#     "101",
# ]

# Consecutive entries are less than 3 days apart until
# 101     (592, 1680)

df_for_ID = df[df["ID"] == "101"].iloc[:591]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "405"]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "272"].iloc[:417]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "201"].iloc[:289]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "203"].iloc[:285]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-2
)
# df[df.ID.isin(interesting_ids)].groupby("ID").apply(
#     lambda df_for_ID: infer_and_plot_for_id(
#         df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-6
#     )
# )


ID: 101
Number of datapoints: 591
Pass 0 - Posteriors' diff for Healthy FEV1 (L): 1.9589499699800248
Pass 0 - Posteriors' diff for Healthy O2 saturation (%): 1.4626076396798535
Pass 1 - Posteriors' diff for Healthy FEV1 (L): 0.13964924761150405
Pass 1 - Posteriors' diff for Healthy O2 saturation (%): 1.3856801960506433e-09
Pass 2 - Posteriors' diff for Healthy FEV1 (L): 3.8856424758407546e-05
Pass 2 - Posteriors' diff for Healthy O2 saturation (%): 8.616907077056024e-14

ID: 405
Number of datapoints: 1035
Pass 0 - Posteriors' diff for Healthy FEV1 (L): 1.9797003031129963
Pass 0 - Posteriors' diff for Healthy O2 saturation (%): 1.4813421090961305
Pass 1 - Posteriors' diff for Healthy FEV1 (L): 0.00015194765806727666
Pass 1 - Posteriors' diff for Healthy O2 saturation (%): 5.484740885679707e-10
Pass 2 - Posteriors' diff for Healthy FEV1 (L): 5.773379713847203e-11
Pass 2 - Posteriors' diff for Healthy O2 saturation (%): 2.6861384605668977e-16

ID: 272
Number of datapoints: 417
Pass 0 - P

/Users/tristan.trebaol/Desktop/PhD/Code/pgmpy/pgmpy/inference/ExactInference.py:1518: RuntimeWarning:

invalid value encountered in divide

/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:480: RuntimeWarning:

invalid value encountered in divide



Pass 1 - Posteriors' diff for Healthy FEV1 (L): nan
Pass 1 - Posteriors' diff for Healthy O2 saturation (%): nan
Pass 2 - Posteriors' diff for Healthy FEV1 (L): nan
Pass 2 - Posteriors' diff for Healthy O2 saturation (%): nan

ID: 203
Number of datapoints: 285
Pass 0 - Posteriors' diff for Healthy FEV1 (L): 1.9514481808636859
Pass 0 - Posteriors' diff for Healthy O2 saturation (%): 1.4675140062663852
Pass 1 - Posteriors' diff for Healthy FEV1 (L): 0.02736567271363481
Pass 1 - Posteriors' diff for Healthy O2 saturation (%): 1.1543560666526968e-09
Pass 2 - Posteriors' diff for Healthy FEV1 (L): 2.811996548310744e-05
Pass 2 - Posteriors' diff for Healthy O2 saturation (%): 1.1882919095750844e-12


In [7]:
import src.data.helpers as dh

dftmp = dh.load_excel(
    f"{dh.get_path_to_main()}/ExcelFiles/BR/Refining_F3/infer_AR_with_two_days_model_O2Sat_FEV1.xlsx",
    # f"{dh.get_path_to_main()}/ExcelFiles/BR/Refining_F3/infer_AR_with_two_days_model_O2Sat_ecFEV1.xlsx",
    [AR.name],
    ["Day"],
).drop(columns=["Unnamed: 0", HO2Sat.name, IA.name, HFEV1.name])

In [13]:
from pgmpy.factors.discrete import TabularCPD

In [14]:
list = [
    TabularCPD(
        HO2Sat.name,
        HO2Sat.card,
        HO2Sat.cpt.reshape(-1, 1),
    ),
    TabularCPD(
        HFEV1.name,
        HFEV1.card,
        HFEV1.cpt.reshape(-1, 1),
    ),
]